In [2]:
from bs4 import BeautifulSoup
import requests
import random

# note for weight value should be 10-120
# checkValueValidation(10, 120, this)

# note for tall value should be 10-120
# checkValueValidation(100, 200, this)

def get_important_information_to_fill_weight_tall():
    pass

def scrape_FitnessWeightHeightMesaures_page(cookies):
    response = requests.get(
        'https://jolearn.jo/index.php?r=EFitness/FitnessWeightHeightMesaures/create',
        cookies=cookies,
    )
    return response.text

def get_grades_data(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')

    # Find the select dropdown by ID
    select_dropdown = soup.find('select', {'id': 'FitnessSchoolSort_ViewCodeNameWHMeasures_GradeID'})

    # Check if the select dropdown is found
    if select_dropdown:
        # Extract options
        grades = []
        for option in select_dropdown.find_all('option'):
            value = option.get('value')
            text = option.get_text(strip=True)
            if "ختر" not in text :
                grades.append({'value': value, 'text': text})
        # print(grades)
        return grades
    
def get_schools_data(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')

    # Find the select dropdown by ID
    select_dropdown = soup.find('select', {'id': 'SchoolID'})

    # Check if the select dropdown is found
    if select_dropdown:
        # Extract options
        schools = []
        for option in select_dropdown.find_all('option'):
            value = option.get('value')
            text = option.get_text(strip=True)
            if len(value):
                schools.append({'value': value, 'text': text})
        # print(schools)
        return schools
    
def scrape_students_ids_html(cookies , schoolId , gradeId):
    data = {
    'viewCodeName': 'ViewCodeNameWHMeasures',
    'SchoolID': schoolId,
    'GradeID': gradeId,
    # 'SchoolID': '4B9EEE39-FC46-4AA7-A33D-1F3FE5E99AC1',
    # 'GradeID': '6363D458-760B-4E5C-8C1D-E15DDA9C13A1',
    }

    response = requests.post(
        'https://jolearn.jo/index.php?r=EFitness/FitnessWeightHeightMesaures/GetListOfAllStudentsPeriods',
        cookies=cookies,
        data=data,
    )
    return response.text

def get_students_ids(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')

    # Find all rows with class 'weightHeightMesauresRow'
    rows = soup.find_all('tr', class_='weightHeightMesauresRow')

    # Extract names and corresponding MesaureStudentID values and store in a list of dictionaries
    data = []
    for row in rows:
        name = row.find('td').get_text(strip=True)
        mesaure_student_id = row.find('input', {'name': 'MesaureStudentID'})['value']
        test_period_id = row.find('input', {'name': 'testPeriodID'})['value']
        data.append({'name': name, 'mesaure_student_id': mesaure_student_id, 'test_period_id': test_period_id})

    return data

def random_request_payload(students_data,w_range=(85, 100) , h_range=(100, 120) ):
    # Define the weight and height ranges
    weight_range = w_range
    height_range = h_range

    # Generate the dictionary
    fitness_data = []
    for item in students_data:
        entry = {
            "StudentID": item['mesaure_student_id'],
            "Weight": str(random.randint(*weight_range)),
            "Height": str(random.randint(*height_range)),
            "TestPeriod": item['test_period_id']
        }
        fitness_data.append(entry)

    # Convert the list of dictionaries to a JSON-like string
    fitness_data_str = {'FitnessWeightHeightMesaures':str(fitness_data).replace("'", '"')}
    return fitness_data_str

# suggestions to make the code better :
# 1. scrape all students data at once then posting them all together
# 2. login to microsoft and get the cookies with requests library instead of selenium library

cookies = {
    'PHPSESSID': 'r86mi4ope2snec78rb8h7emk6d',
    'ARRAffinity': 'afb7226a31ec5e26360c36e5e18645c7826aa5dd89a26f31b33c5ca2e1c5d7bd',
    'ARRAffinitySameSite': 'afb7226a31ec5e26360c36e5e18645c7826aa5dd89a26f31b33c5ca2e1c5d7bd',
}

FitnessWeightHeightMesaures_page = scrape_FitnessWeightHeightMesaures_page(cookies)
schools = [ i['value'] for i in get_schools_data(FitnessWeightHeightMesaures_page)][0]
grades = get_grades_data(FitnessWeightHeightMesaures_page)

for grade in grades:
    html_text = scrape_students_ids_html(cookies , schools , grade['value'])
    students_data = [i for i in get_students_ids(html_text)]
    request_payload = random_request_payload(students_data)
    
    response = requests.post(
        'https://jolearn.jo/index.php?r=EFitness/FitnessWeightHeightMesaures/create',
        cookies=cookies,
        data=request_payload
    )
    print(" تم تعبئة الصف بنجاح " + grade['text'])
    print(response.status_code )
    

 تم تعبئة الصف بنجاح الصف السابع
200
 تم تعبئة الصف بنجاح الصف الثامن
200
 تم تعبئة الصف بنجاح الصف التاسع
200
 تم تعبئة الصف بنجاح الصف العاشر
200
 تم تعبئة الصف بنجاح الصف الحادي عشر الأدبي
200
 تم تعبئة الصف بنجاح الصف الحادي عشر العلمي
200
 تم تعبئة الصف بنجاح الصف الثاني عشر الأدبي
200
 تم تعبئة الصف بنجاح الصف الثاني عشر العلمي
200


In [18]:
controlled_ranodmness = {'سابع' : [(140,150) ,(48,55)] ,
                        'ثامن' : [(148,156) ,(55,62)] ,
                        'تاسع' : [(152,160) ,(60,69)] ,
                        'عاشر' : [(157,164) ,(64,71)] ,
                        'حادي عشر' : [(163,170) ,(64,75)] ,
                        'ثاني عشر' : [(163,175) ,(65,78)]}

for grade in grades:
    for item in controlled_ranodmness:
        if item in grade['text']:
            h_range , w_range = controlled_ranodmness[item]
            request_payload = random_request_payload(students_data ,h_range=h_range,w_range=w_range)
            print(grade)
        else:
            print('not found')
    # grade['text']

{'value': '6363D458-760B-4E5C-8C1D-E15DDA9C13A1', 'text': 'الصف السابع'}
not found
not found
not found
not found
not found
not found
{'value': '78E1D9DB-5E83-4461-BDD7-BDD0733B1C5C', 'text': 'الصف الثامن'}
not found
not found
not found
not found
not found
not found
{'value': '42B27C7A-37D1-41AD-B8E3-4E1C999EC14A', 'text': 'الصف التاسع'}
not found
not found
not found
not found
not found
not found
{'value': '9A6D6429-39C2-458E-89CB-4ED9960DF4B9', 'text': 'الصف العاشر'}
not found
not found
not found
not found
not found
not found
{'value': 'E92F824F-C029-40CA-B225-58AEAE3684E4', 'text': 'الصف الحادي عشر الأدبي'}
not found
not found
not found
not found
not found
{'value': '0D97EBB9-0C4C-4D5B-9E47-AD638C73BA74', 'text': 'الصف الحادي عشر العلمي'}
not found
not found
not found
not found
not found
not found
{'value': '534CCEEE-9BB4-4C13-BB63-D04FDAC930C7', 'text': 'الصف الثاني عشر الأدبي'}
not found
not found
not found
not found
not found
{'value': '4DF525CD-7420-4D81-AFC9-3D1784B8D8E4', 'text'